# The Paillier Cryptosystem

## Key generation steps

* Generate two random primes $p$ and $q$
* Calculate $N$, the product of $p$ and $q$
* if $N$ and $\phi(N)$ have common factors, go back to first step and generate new $p$ and $q$
* if $N$ and $\phi(N)$ don't share common factors (i.e. gcd is 1) then:
    * calculate $l$, the least common multiple of $p-1$ and $q-1$
    * calculate $N^2$
    * draw a random number $g$ in between 1 and $N^2$
    * calculate $\mu$ as the inverse of $L(g^l \textit{mod }N^2, N)$ in modulo $N$ where $L(x, n)=(x-1)/n$
    
The public key is ($N$, $g$) and the private key is ($N$, $l$, $\mu$) 

### Drawing two prime numbers

In [1]:
from crypto import RandomPrime
from crypto import xgcd

size_bits = 16

p = RandomPrime(size_bits, m=40)
q = RandomPrime(size_bits, m=40)

while p==q:
    q = RandomPrime(size_bits, m=40)

N = p*q
gcd, _, _ = xgcd(N, (p-1)*(q-1))

print(f"p = {p}")
print(f"q = {q}")
print(f"gcd(N, (p-1)*(q-1))={gcd}")

p = 53327
q = 56533
gcd(N, (p-1)*(q-1))=1


### Calculating l, g and mu

In [2]:
from random import randrange
from crypto import LCM, InverseMod

def _L(x, n):
    return (x-1)//n

l = LCM(p-1, q-1)
nsq = N*N
g = randrange(1, nsq)
mu = InverseMod(_L(pow(g, l, nsq), N), N)

print(f"l = {l}")
print(f"N^2 = {nsq}")
print(f"mu = {mu}")

l = 215330388
N^2 = 9088628874800854681
mu = 2106565611


In [3]:
PublicKey = (N, g)
PrivateKey = (N, l, mu)

print(f"PublicKey = (N, g) = ({N}, {g})")
print(f"PrivateKey = (N, l, mu) = ({N}, {l}, {mu})")

PublicKey = (N, g) = (3014735291, 1698245462146605806)
PrivateKey = (N, l, mu) = (3014735291, 215330388, 2106565611)


## Encryption function

Take the public key ($N$, $g$) and the message you want to send $m$. Find a random number $r<N$ such that it has no common factors with $N$. Then compute the ciphertext $c$ as:

$$c = g^{m}*r^{N}(\text{mod }N^2)$$

In [4]:
m = randrange(0, N)

N, g = PublicKey[0], PublicKey[1]
gcd = 2

while gcd!=1:
    r = randrange(1, N)
    gcd, _, _ = xgcd(r, N)

c = pow(g, m, N*N)*pow(r, N, N*N)%(N*N)

print(f"m: {m}")
print(f"c: {c}")

m: 2471308924
c: 4332288696113767125


## Decryption function

Take the private key ($N$, $l$, $\mu$) and the ciphertext $c$ and compute:

$$m = L(c^l(\text{mod }N^2), N)*\mu (\text{mod }N)$$

In [5]:
N, l, mu = PrivateKey[0], PrivateKey[1], PrivateKey[2]
m2 = _L(pow(c, l, N*N), N)*mu%N

print(f"Recovered message: {m2}")

Recovered message: 2471308924


In [6]:
from typing import Tuple


def PaillierKeyGenerator(size: int = 64):
    '''
    Implementation of Paillier Cryptosystem
    This function generates p                                                                                                                                                                                                                                                                                                                        .
    ublic and private keys
    Input:
        size: size in bits of the field
    Output:
        PublicKey: (n, g)
        PrivateKey: (n, l, mu)
    '''
    
    gcd = 2
    while gcd!=1:
        p = RandomPrime(size, 40)
        q = RandomPrime(size, 40)
        N = p*q

        gcd, _, _ = xgcd(N, (p-1)*(q-1))

        if gcd==1:
            l = LCM(p-1, q-1)
            nsq = N*N
            g = randrange(1, nsq)
            mu = InverseMod(_L(pow(g, l, nsq), N), N)

    return (N, g), (N, l, mu)

def PaillierEncrypt(m: int, PublicKey: Tuple[int, int]):
    '''
    Encrypts a message m using the Paillier public key
    Input:
        m: message (An integer message) (mod n)
        PublicKey: A tuple (N, g)
    Output:
        c: Encrypted message
    '''
    N, g = PublicKey[0], PublicKey[1]
    gcd = 2
    while gcd!=1:
        r = randrange(1, N)
        gcd, _, _ = xgcd(r, N)

    return pow(g, m, N*N)*pow(r, N, N*N)%(N*N)


def PaillierDecrypt(c: int, PrivateKey: Tuple[int, int, int]):
    '''
    Decrypts a ciphertext m using the Paillier private key
    Input:
        m: message (An integer message) (mod n)
        PublicKey: A tuple (n, l, mu)
    Output:
        m: Decrypted message
    '''
    N, l, mu = PrivateKey[0], PrivateKey[1], PrivateKey[2]
    return _L(pow(c, l, N*N), N)*mu%N

In [7]:
PublicKey, PrivateKey = PaillierKeyGenerator(32)

print(f"PublicKey = {PublicKey}")
print(f"PrivateKey = {PrivateKey}")

PublicKey = (9267623274722409313, 83695160699575164744975302747966218772)
PrivateKey = (9267623274722409313, 1544603878095019350, 1566882554582969803)


In [52]:
m = randrange(0, N)
c = PaillierEncrypt(m, PublicKey)
m2 = PaillierDecrypt(c, PrivateKey)

print(f"message: {m}")
print(f"ciphertext: {c}")
print(f"recovered_message: {m2}")

message: 1843263063
ciphertext: 35509801174405282742644239699714465305
recovered_message: 1843263063
